In [35]:
import pandas as pd

# Load data from CSV files with tab ('\\t') delimiter
sheet1 = pd.read_csv('guide_rna_with_ucsc_links2.csv', delimiter='\t')
sheet2 = pd.read_csv('guide_rna_with_ucsc_links3.csv', delimiter='\t')
sheet3 = pd.read_csv('guide_rna_with_ucsc_links4.csv', delimiter='\t')

# Ensure column names are consistent
sheet1.columns = ['Guide_Name', 'Transcript_Name', 'Guide_Sequence']
sheet2.columns = ['Guide_Name', 'Transcript_Name', 'Guide_Sequence']
sheet3.columns = ['Guide_Name', 'Transcript_Name', 'Guide_Sequence']

# Extract Gene_Name from Guide_Name
all_data['Gene_Name'] = all_data['Guide_Name'].str.split('_').str[0]

# Define a dictionary mapping gene names to chromosome coordinates and strands
gene_coordinates = {
    'CCND1': 'chr11:69641156-69654474(+)',
    'PRNCR1': 'chr8:127079874-127092600(+)',
    'LINC00299': 'chr2:7988683-8488284(-)',
    'KCNQ1OT1': 'chr11:2597308-2700003(-)',
    'HOTAIRM1': 'chr7:27095647-27100265(+)',
    'PTENP1': 'chr9:33675437-33676651(-)',
    'HOTTIP': 'chr7:27198575-27207259(+)',
    'TUG1': 'chr22:30969245-30979395(+)',
    'HAR1A': 'chr20:63102205-63104386(+)',
    'RMST': 'chr12:97430884-97598415(+)',
    'UCA1': 'chr19:15828206-15836328(+)',
    'GHET1': 'chr7:148987527-148989432(+)',
    'NRON': 'chr9:126408041-126408410(-)',
    'TINCR': 'chr19:5558167-5578349(-)',
    'SENCR': 'chr11:128691664-128696023(-)',
    'XACT': 'chrX:113616300-114059289(-)',
    'CRNDE': 'chr16:54845189-54929189(-)',
    'LINC00467': 'chr1:211382736-211435570(+)'
}

# Add new columns with chromosome, start, end, and strand
all_data['Chromosome'] = all_data['Gene_Name'].map(gene_coordinates).str.split(':').str[0]
all_data['Start'] = all_data['Gene_Name'].map(gene_coordinates).str.split(':').str[1].str.split('-').str[0]
all_data['End'] = all_data['Gene_Name'].map(gene_coordinates).str.split('-').str[1].str.split('(').str[0]
all_data['Strand'] = all_data['Gene_Name'].map(gene_coordinates).str.split('(').str[1].str.split(')').str[0]

# Displaying unique gene names
unique_gene_names = all_data['Gene_Name'].unique()
print("Unique Gene Names:")
print(unique_gene_names)

# Save the unique gene names to a new CSV file
unique_gene_names_df = pd.DataFrame(unique_gene_names, columns=['Unique_Gene_Names'])
unique_gene_names_file = 'unique_gene_names.csv'
unique_gene_names_df.to_csv(unique_gene_names_file, index=False)
print(f"Unique gene names saved to {unique_gene_names_file}")

# Displaying the new DataFrame with additional columns
print("\nNew DataFrame with Transcript_Name, Guide_Sequence, Gene_Name, Chromosome, Start, End, and Strand:")
new_dataframe = all_data[['Transcript_Name', 'Guide_Sequence', 'Gene_Name', 'Chromosome', 'Start', 'End', 'Strand']]
print(new_dataframe)

# Save the new DataFrame with additional columns to a new CSV file
new_dataframe_file = 'new_dataframe_with_columns.csv'
new_dataframe.to_csv(new_dataframe_file, index=False)
print(f"New DataFrame with selected columns saved to {new_dataframe_file}")

Unique Gene Names:
['RMST' 'HAR1A' 'TUG1' 'HOTTIP' 'PTENP1' 'HOTAIRM1' 'KCNQ1OT1' 'LINC00299'
 'XACT' 'PRNCR1' 'CCND1' 'LINC00467' 'NRON' 'CRNDE' 'GHET1' 'SENCR'
 'TINCR' 'UCA1']
Unique gene names saved to unique_gene_names.csv

New DataFrame with Transcript_Name, Guide_Sequence, Gene_Name, Chromosome, Start, End, and Strand:
       Transcript_Name                Guide_Sequence Gene_Name Chromosome  \
0      ENST00000538559  GTGAGAAACAAGGCATTGCCTATAACTG      RMST      chr12   
1      ENST00000538559  GGTGAGAAACAAGGCATTGCCTATAACT      RMST      chr12   
2      ENST00000538559  TGGTGAGAAACAAGGCATTGCCTATAAC      RMST      chr12   
3      ENST00000538559  GTGGTGAGAAACAAGGCATTGCCTATAA      RMST      chr12   
4      ENST00000538559  CGTGGTGAGAAACAAGGCATTGCCTATA      RMST      chr12   
...                ...                           ...       ...        ...   
33965  ENST00000649506  TAATAGATGGATGGACCCTGGATTGCTT      XACT       chrX   
33966  ENST00000649506  TTAATAGATGGATGGACCCTGGATTGCT    

In [36]:
import pandas as pd
import urllib.parse

def generate_ucsc_link_with_guide_track(chromosome, start, guide_sequence, genome='hg38'):
    """
    Generates a UCSC Genome Browser link with a custom track to visualize the guide RNA sequence.
    The 'start' parameter should be the start position of the guide RNA.
    The end is calculated based on the length of the guide sequence.
    """
    if not chromosome.lower().startswith('chr'):
        chromosome = f'chr{chromosome}'
    
    start = int(start)
    end = start + len(guide_sequence)  # Calculate end based on the guide sequence length
    
    # Create a BED format custom track for the guide sequence
    bed_data = f"{chromosome}\t{start}\t{end}\t{guide_sequence}\t0\t+"
    encoded_bed = urllib.parse.quote(bed_data)
    
    base_url = "https://genome.ucsc.edu/cgi-bin/hgTracks"
    url = (f"{base_url}?db={genome}&position={chromosome}%3A{start}-{end}"
           f"&hgct_customText=track%20name=guide_RNA%20description=%22Guide%20RNA%22"
           f"%20visibility=pack%0A{encoded_bed}")
    
    return url

def process_file_with_guide_visualization(input_file, output_file):
    # Load the file
    if input_file.endswith('.csv'):
        data = pd.read_csv(input_file)
    elif input_file.endswith('.xlsx'):
        data = pd.read_excel(input_file)
    else:
        raise ValueError("Unsupported file type. Please use a CSV or Excel file.")
    
    # Validate columns
    expected_columns = ['Transcript_Name', 'Guide_Sequence', 'Gene_Name', 'Chromosome', 'Start', 'End', 'Strand']
    column_map = {col.lower(): col for col in data.columns}
    missing_columns = [col for col in expected_columns if col.lower() not in column_map]
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
    actual_columns = {col: column_map[col.lower()] for col in expected_columns}
    
    # Generate UCSC links with guide RNA custom tracks
    data['UCSC_URL'] = data.apply(
        lambda row: generate_ucsc_link_with_guide_track(
            row[actual_columns['Chromosome']], 
            row[actual_columns['Start']],  # Start position of the guide RNA
            row[actual_columns['Guide_Sequence']]  # Use guide RNA sequence length to calculate end
        ), axis=1
    )
    
    # Save the output
    output_columns = [actual_columns['Transcript_Name'], actual_columns['Guide_Sequence'], 
                      actual_columns['Gene_Name'], actual_columns['Chromosome'], 
                      actual_columns['Start'], actual_columns['End'], 
                      actual_columns['Strand'], 'UCSC_URL']
    
    data[output_columns].to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

# Example usage:
input_file = 'new_dataframe_with_columns.csv'  # Replace with your actual file path
output_file = 'output_file_with_visualized_guide_links.csv'  # Replace with your desired output file path

process_file_with_guide_visualization(input_file, output_file)

Results saved to output_file_with_visualized_guide_links.csv
